# Lecture 5 - Inconsistent Data Entry

Sửa lỗi chính tả một cách hiệu quả trong dữ liệu của bạn.

Trong sổ tay này, chúng ta sẽ học cách làm sạch các mục văn bản không nhất quán.

Hãy bắt đầu thôi!


## Thiết lập môi trường của chúng ta

Điều đầu tiên chúng ta cần làm là tải các thư viện và bộ dữ liệu mà chúng ta sẽ sử dụng.

In [19]:
# Các mô-đun chúng ta sẽ sử dụng
import pandas as pd
import numpy as np

# Các mô-đun hữu ích
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

# Đọc toàn bộ dữ liệu của chúng ta
professors = pd.read_csv("pakistan_intellectual_capital.csv")

# Thiết lập hạt giống để tái lập kết quả
np.random.seed(0)

## Tiến hành tiền xử lý văn bản ban đầu

Chúng ta sẽ bắt đầu bằng cách nhìn qua vài dòng đầu tiên của dữ liệu.


In [20]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


Giả sử chúng ta muốn làm sạch cột "Country" để đảm bảo không có sự không nhất quán trong việc nhập liệu. Chúng ta có thể đi qua và kiểm tra từng dòng bằng tay, dĩ nhiên, và chỉnh sửa thủ công các sự không nhất quán khi tìm thấy chúng. Tuy nhiên, có một cách hiệu quả hơn để làm việc này!

In [21]:
# Lấy tất cả các giá trị duy nhất trong cột 'Country'
countries = professors['Country'].unique()

# Sắp xếp chúng theo thứ tự bảng chữ cái và sau đó xem kỹ hơn
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

Chỉ cần nhìn vào đây, tôi có thể thấy một số vấn đề do nhập liệu không nhất quán: ví dụ như ' Germany' và 'germany', hay ' New Zealand' và 'New Zealand'.

Điều đầu tiên tôi sẽ làm là chuyển tất cả thành chữ thường (tôi có thể chuyển lại nếu muốn) và loại bỏ bất kỳ khoảng trắng nào ở đầu và cuối của các ô. Các sự không nhất quán trong việc viết hoa và khoảng trắng thừa là rất phổ biến trong dữ liệu văn bản, và bạn có thể sửa khoảng 80% các sự không nhất quán trong việc nhập liệu văn bản của mình bằng cách làm điều này.

In [22]:
# Chuyển tất cả thành chữ thường
professors['Country'] = professors['Country'].str.lower()

# Loại bỏ khoảng trắng thừa
professors['Country'] = professors['Country'].str.strip()

Tiếp theo, chúng ta sẽ xử lý các sự không nhất quán phức tạp hơn.

## Sử dụng fuzzy matching để sửa các lỗi nhập liệu không nhất quán

Được rồi, hãy xem lại cột 'Country' và xem liệu còn có bất kỳ công việc làm sạch dữ liệu nào cần làm nữa không.

In [23]:
# Lấy tất cả các giá trị duy nhất trong cột 'Country'
countries = professors['Country'].unique()

# Sắp xếp chúng theo thứ tự bảng chữ cái và sau đó xem kỹ hơn
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

Có vẻ như vẫn còn một sự không nhất quán khác: 'southkorea' và 'south korea' nên là giống nhau.

Chúng ta sẽ sử dụng gói [fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy) để giúp xác định các chuỗi nào gần với nhau nhất. Bộ dữ liệu này đủ nhỏ để chúng ta có thể sửa lỗi bằng tay, nhưng cách tiếp cận này không thể mở rộng tốt. (Bạn có muốn sửa một nghìn lỗi bằng tay không? Còn mười nghìn lỗi thì sao? Tự động hóa mọi thứ càng sớm càng tốt thường là một ý tưởng hay. Thêm vào đó, nó cũng thú vị!)

> **Fuzzy matching:** Quá trình tự động tìm các chuỗi văn bản tương tự với chuỗi mục tiêu. Nói chung, một chuỗi được xem là "gần hơn" với một chuỗi khác nếu bạn cần thay đổi ít ký tự hơn khi biến đổi một chuỗi thành chuỗi kia. Vì vậy, "apple" và "snapple" chỉ cách nhau hai thay đổi (thêm "s" và "n"), trong khi "in" và "on" chỉ cách nhau một thay đổi (thay "i" bằng "o"). Bạn sẽ không thể lúc nào cũng dựa vào fuzzy matching 100%, nhưng nó thường sẽ giúp bạn tiết kiệm ít nhất một chút thời gian.

Fuzzywuzzy trả về một tỷ lệ cho hai chuỗi. Tỷ lệ càng gần 100, khoảng cách chỉnh sửa giữa hai chuỗi càng nhỏ. Ở đây, chúng ta sẽ lấy mười chuỗi trong danh sách các thành phố có khoảng cách gần nhất với "south korea".

In [24]:
# Lấy 10 kết quả gần nhất với "south korea"
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# Xem qua các kết quả
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('ireland', 33),
 ('portugal', 32),
 ('singapore', 30),
 ('netherland', 29),
 ('macau', 25),
 ('usofa', 25)]

Chúng ta có thể thấy rằng hai mục trong danh sách các thành phố rất gần với "south korea": "south korea" và "southkorea". Hãy thay thế tất cả các dòng trong cột "Country" có tỷ lệ > 47 bằng "south korea".

Để làm điều này, tôi sẽ viết một hàm. (Viết một hàm chung có thể tái sử dụng là một ý tưởng hay nếu bạn nghĩ rằng mình có thể phải làm một nhiệm vụ cụ thể nhiều hơn một hoặc hai lần. Điều này giúp bạn không phải sao chép và dán mã quá thường xuyên, tiết kiệm thời gian và có thể giúp ngăn ngừa sai sót.)

In [25]:
# Hàm thay thế các dòng trong cột được cung cấp của dataframe với các chuỗi khớp với chuỗi đã cung cấp có tỷ lệ lớn hơn tỷ lệ tối thiểu
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # Lấy danh sách các chuỗi duy nhất
    strings = df[column].unique()
    
    # Lấy 10 kết quả gần nhất với chuỗi nhập vào
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Chỉ lấy các kết quả có tỷ lệ > 47
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Lấy các dòng có các chuỗi khớp trong dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Thay thế tất cả các dòng có chuỗi khớp bằng chuỗi nhập vào
    df.loc[rows_with_matches, column] = string_to_match
    
    # Thông báo hàm đã thực hiện xong
    print("All done!")


Giờ chúng ta đã có một hàm, hãy thử nghiệm nó!


In [26]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

All done!


Và bây giờ, hãy kiểm tra lại các giá trị duy nhất trong cột "Country" và đảm bảo rằng chúng ta đã làm sạch "south korea" một cách chính xác.

In [27]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

Tuyệt vời! Giờ chúng ta chỉ còn "south korea" trong dataframe và chúng ta không cần phải thay đổi gì bằng tay.

## Your turn!

Therre are still some inconsistencies in the dataset.  [**Continue cleaning the data**](https://www.kaggle.com/kernels/fork/10824407) in a final exercise.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/data-cleaning/discussion) to chat with other learners.*